In [1]:
import pandas as pd
import pyspark
sc = pyspark.SparkContext('local[*]')

In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv')
postulaciones = pd.read_csv('Sources/fiuba_4_postulaciones_concat.csv')

In [3]:
postulantes.drop(['Unnamed: 0.1'],axis=1,inplace=True)
postulantes = postulantes[['idpostulante','fechanacimiento','sexo']]
postulantes.head()

,idpostulante,fechanacimiento,sexo
0,0z5Dmrd,53.0,MASC
1,0z5JW1r,47.0,MASC
2,0z5VvGv,31.0,MASC
3,0zB01pE,29.0,MASC
4,0zB026d,40.0,FEM


In [4]:
postulaciones.drop(['Unnamed: 0','fechapostulacion'],axis=1,inplace=True)
postulaciones.head()

,idaviso,idpostulante
0,1112248724,NjlD
1,1112286523,ZaO5
2,1112272060,ZaO5
3,1112288401,ZaO5
4,1112300563,ZaO5


In [5]:
postulaciones = postulaciones.merge(postulantes, on='idpostulante',how='right')
postulaciones.head()

,idaviso,idpostulante,fechanacimiento,sexo
0,1.112249e+09,NjlD,37.0,FEM
1,1.112287e+09,ZaO5,40.0,FEM
2,1.112272e+09,ZaO5,40.0,FEM
3,1.112288e+09,ZaO5,40.0,FEM
4,1.112301e+09,ZaO5,40.0,FEM


In [6]:
prom_edad = postulaciones.groupby(['idaviso']).mean()

In [7]:
prom_edad.head()

,fechanacimiento
idaviso,
8725750.0,32.165746
11740890.0,29.000000
12543760.0,29.985859
12812680.0,25.115000
17903700.0,33.341176


In [8]:
sexo_mayoritario = postulaciones.groupby(['idaviso','sexo'],as_index=False).count()
sexo_mayoritario.head()

,idaviso,sexo,idpostulante,fechanacimiento
0,8.725750e+06,FEM,70,70
1,8.725750e+06,MASC,111,111
2,8.725750e+06,NO_DECLARA,6,0
3,1.174089e+07,FEM,10,10
4,1.174089e+07,MASC,9,9
5,1.254376e+07,FEM,861,860
6,1.254376e+07,MASC,623,623
7,1.254376e+07,NO_DECLARA,36,2
8,1.281268e+07,FEM,180,180
9,1.281268e+07,MASC,18,18


In [9]:
sexo_mayoritario = sexo_mayoritario.join(pd.get_dummies(sexo_mayoritario['sexo']))

In [12]:
sexo_mayoritario['FEM'] = sexo_mayoritario.apply(lambda x: x['idpostulante'] if x['sexo']=='FEM' else 0,axis=1)
sexo_mayoritario['MASC'] = sexo_mayoritario.apply(lambda x: x['idpostulante'] if x['sexo']=='MASC' else 0,axis=1)
sexo_mayoritario['NO_DECLARA'] = sexo_mayoritario.apply(lambda x: x['idpostulante'] if x['sexo']=='NO_DECLARA' else 0,axis=1)

In [14]:
sexo_mayoritario = sexo_mayoritario[['idaviso','FEM','MASC','NO_DECLARA']]
sexo_mayoritario = sexo_mayoritario.groupby(['idaviso'])['FEM','MASC','NO_DECLARA'].sum()
sexo_mayoritario.head()

,FEM,MASC,NO_DECLARA
idaviso,,,
8.725750e+06,70,111,6
1.174089e+07,10,9,0
1.254376e+07,861,623,36
1.281268e+07,180,18,7
1.790370e+07,151,19,5
1.000132e+09,3,15,0
1.000139e+09,240,285,15
1.000365e+09,302,106,2
1.000366e+09,43,114,4


In [16]:
def mayor(F,M,N):
    suma = F+M+N
    if (F/suma > 1.3 * M/suma):
        return 'FEM'
    if (M/suma > 1.3 * F/suma):
        return 'MASC'
    return 'NULL'

In [17]:
sexo_mayoritario['TendenciaSexo'] = sexo_mayoritario.apply(lambda x: mayor(x['FEM'],x['MASC'],x['NO_DECLARA']), axis=1)

In [18]:
sexo_mayoritario.head()

,FEM,MASC,NO_DECLARA,TendenciaSexo
idaviso,,,,
8725750.0,70,111,6,MASC
11740890.0,10,9,0,NULL
12543760.0,861,623,36,FEM
12812680.0,180,18,7,FEM
17903700.0,151,19,5,FEM


In [19]:
sexo_mayoritario.reset_index(inplace=True)

In [21]:
sexo_mayoritario.head()

,idaviso,FEM,MASC,NO_DECLARA,TendenciaSexo
0,8725750.0,70,111,6,MASC
1,11740890.0,10,9,0,NULL
2,12543760.0,861,623,36,FEM
3,12812680.0,180,18,7,FEM
4,17903700.0,151,19,5,FEM


In [22]:
prom_edad.reset_index(inplace=True)
prom_edad.head()

,index,idaviso,fechanacimiento
0,0,8725750.0,32.165746
1,1,11740890.0,29.000000
2,2,12543760.0,29.985859
3,3,12812680.0,25.115000
4,4,17903700.0,33.341176


In [23]:
sexo_mayoritario = sexo_mayoritario.merge(prom_edad,on='idaviso',how='left')
sexo_mayoritario.head()

,idaviso,FEM,MASC,NO_DECLARA,TendenciaSexo,index,fechanacimiento
0,8725750.0,70,111,6,MASC,0,32.165746
1,11740890.0,10,9,0,NULL,1,29.000000
2,12543760.0,861,623,36,FEM,2,29.985859
3,12812680.0,180,18,7,FEM,3,25.115000
4,17903700.0,151,19,5,FEM,4,33.341176


In [24]:
sexo_mayoritario = sexo_mayoritario[['idaviso','FEM','MASC','NO_DECLARA','TendenciaSexo','fechanacimiento']]
sexo_mayoritario.rename(columns={'fechanacimiento':'prom_edad'},inplace=True)
sexo_mayoritario.head()

,idaviso,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad
0,8725750.0,70,111,6,MASC,32.165746
1,11740890.0,10,9,0,NULL,29.000000
2,12543760.0,861,623,36,FEM,29.985859
3,12812680.0,180,18,7,FEM,25.115000
4,17903700.0,151,19,5,FEM,33.341176


In [ ]:
sexo_mayoritario.to_csv('Sources/mas_features.csv')

In [25]:
sexo_mayoritario.shape

(20231, 6)